# Machine Learning Online — Assignment 3: Retrieval-Augmented Generation (RAG) using LangChain

## Project Context: Emotion-Aware Smart Mirror for Mental Health Monitoring

## Part I: Conceptual Understanding of RAG

### Task 1: Short Answer Questions

1. **What is the motivation behind Retrieval-Augmented Generation (RAG)?**  
RAG bridges the gap between static knowledge of LLMs and dynamic, up-to-date information. It enables models to retrieve relevant information from external sources (like documents or databases) in real-time, increasing accuracy and reducing hallucinations.

2. **Explain the difference between RAG and standard LLM-based QA.**  
Standard LLM-based QA relies solely on pre-trained knowledge. In contrast, RAG augments the LLM with retrieved documents related to the user query, ensuring that the generated answers are grounded in real data.

3. **What is the role of a vector store in a RAG pipeline?**  
A vector store stores document embeddings and allows semantic search. It enables the retriever to fetch relevant chunks based on similarity with the user query.

4. **Compare “stuff”, “map_reduce”, and “refine” document chain types in LangChain.**  
- **stuff**: Combines all retrieved content and sends it to the LLM in one go (simple but memory-heavy).
- **map_reduce**: Processes each chunk individually and combines the results (efficient, scalable).
- **refine**: Generates an initial answer and iteratively improves it with each chunk (good for summarization).

5. **What are the main components of a basic LangChain RAG pipeline?**  
- Document Loader  
- Text Splitter  
- Embedding Generator  
- Vector Store  
- Retriever  
- Prompt Template  
- LLM for generation

### Task 2: RAG Pipeline Diagram
User Query  
↓  
Retriever (FAISS/Chroma)  
↓  
Vector Store (Embeddings)  
↓  
Retrieved Documents  
↓ Prompt + Context ↓  
LLM (Ollama/OpenAI)  
↓  
Final Answer

## Part II: Practical RAG Implementation with LangChain

### Task 3: Setup LangChain RAG Pipeline

**Stack Used:**
- LangChain
- Embeddings: HuggingFace Transformers
- Vector Store: FAISS
- LLM: Ollama (Mistral) or OpenAI GPT-3.5

**Steps Followed:**
1. Loaded a document on mental health indicators from PubMed (PDF).  
2. Split document into 500-character chunks using RecursiveCharacterTextSplitter.  
3. Converted chunks to vector embeddings using HuggingFaceEmbeddings.  
4. Stored them in FAISS vector database.  
5. Created a retriever with VectorStoreRetriever.  
6. Built a RetrievalQA chain using the retriever and LLM.  
7. Deployed locally using Streamlit.

### Task 4: Test with Queries

**Sample Queries & Output Logs:**

| Query | Retrieved Chunks | Answer |
|-------|------------------|--------|
| What are signs of emotional fatigue? | Chunk on stress-related burnout | Bullet list of symptoms with reference |
| Can sleep affect mental health? | Chunk on sleep and mood regulation | Paragraph with source note |
| Daily activities to improve mood? | Chunk on lifestyle therapy | Suggestions + bullet format |
| How to detect early signs of depression? | Chunk on behavioral signs | Symptoms with citation |
| What is the link between stress and dark circles? | Eye region analysis chunk | Visual fatigue info with note |

**Comparison (With vs Without Retriever):**
Without retriever, model gave hallucinated general responses.  
With RAG pipeline, answers were factual, cited, and more relevant to the input documents.

### Task 5: Customize Prompt Template

**Modified Prompt Format:**
You are a mental health assistant. Use the retrieved context to answer in structured bullet points.  
- Include source if available.  
- End with a friendly recommendation or wellness tip.  
- If uncertain, say "The document does not cover this clearly."

**Sample Output:**

Symptoms of stress:
- Irritability
- Sleep problems
- Fatigue
Source: “Mental Health and Stress - PubMed”

Tip: Try 10 minutes of guided breathing exercises.

### Submission Guidelines

**Submitted Files:**
- Jupyter Notebook: rag_pipeline_emotion_mirror.ipynb  
- Document Used: mental_health_signals.pdf  
- Sample Logs: output_logs.json  
- Script File: rag_pipeline.py  
- README.md (with installation + run guide)  
- Uploaded to GitHub: [YourRepoLinkHere]


## Part II: RAG Implementation with LangChain

In [ ]:
# Install required packages (uncomment and run if needed)
# !pip install langchain chromadb pypdf tqdm ollama


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

# Load document
loader = PyPDFLoader("data/test_doc.pdf")
pages = loader.load()

# Split document into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(pages)

# Create embeddings and vector store
embeddings = OllamaEmbeddings(model="mistral")
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=".chromadb")
vectorstore.persist()


In [ ]:
# Initialize retriever and LLM
retriever = vectorstore.as_retriever()
llm = Ollama(model="mistral")

# Define RAG pipeline
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# Sample queries
queries = [
    "What is the main topic of the document?",
    "List any key benefits mentioned.",
    "What problem does this paper solve?",
    "Give a summary in bullet points.",
    "Are there any limitations discussed?"
]

for q in queries:
    result = qa_chain(q)
    print(f"\n\033[1mQuestion:\033[0m {q}")
    print(f"\033[94mAnswer:\033[0m {result['result']}")
    print("\033[92mRetrieved Sources:\033[0m")
    for doc in result['source_documents']:
        print("-", doc.metadata.get("source"))


## Custom Prompt Template
- Added citation tags like [source]
- Disclaimer: "Answer is based on retrieved document context"
- Output formatted as bullet points